<a href="https://colab.research.google.com/github/vamsikrishna00466/stock-market-anlalysis/blob/main/HEIKIN_ASHI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mysql-connector-python-rf

     |████████████████████████████████| 11.9MB 5.9MB/s 
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp37-cp37m-linux_x86_64.whl size=249456 sha256=0a4da8d2aa701208963d9caa29d1ee03040bc059edfaa913872c51211f8a7c3e
  Stored in directory: /root/.cache/pip/wheels/87/58/fb/d95c84fad7e1bebfed324c13e107ebb08e1997c9226532859a
Successfully built mysql-connector-python-rf


In [2]:
#importing relevant libraries
import os
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import operator
import decimal
import time
import mysql.connector as mysql
from math import floor
import itertools
import warnings
warnings.filterwarnings('ignore')

In [4]:
df=df[['ins_date', 'open', 'high', 'low', 'close']]

In [5]:
df_original=df.copy()

In [6]:
df.tail()

,ins_date,open,high,low,close
1145177,2021-03-19 09:57:00,34092.80,34098.50,34084.20,34089.15
1145178,2021-03-19 09:58:00,34085.75,34103.70,34083.80,34103.70
1145179,2021-03-19 09:58:00,34085.75,34103.70,34083.80,34103.70
1145180,2021-03-19 09:59:00,34096.90,34124.30,34095.50,34115.10
1145181,2021-03-19 09:59:00,34096.90,34124.30,34095.50,34115.10


In [7]:
#remove duplicates
df1 = df.drop_duplicates(keep='last')
df1=df1.reset_index()
df1=df1[['ins_date','open','high','low','close']]

In [8]:
df1.tail()

,ins_date,open,high,low,close
572588,2021-03-19 09:55:00,34050.90,34070.15,34038.95,34069.80
572589,2021-03-19 09:56:00,34069.10,34097.40,34067.20,34094.35
572590,2021-03-19 09:57:00,34092.80,34098.50,34084.20,34089.15
572591,2021-03-19 09:58:00,34085.75,34103.70,34083.80,34103.70
572592,2021-03-19 09:59:00,34096.90,34124.30,34095.50,34115.10


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572593 entries, 0 to 572592
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   ins_date  572593 non-null  datetime64[ns]
 1   open      572593 non-null  object        
 2   high      572593 non-null  object        
 3   low       572593 non-null  object        
 4   close     572593 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 21.8+ MB


In [10]:
#change the time zone
df1['ins_date'] = df1['ins_date'].dt.tz_localize('GMT').dt.tz_convert('Asia/Kolkata')

In [11]:
df1.head()

,ins_date,open,high,low,close
0,2015-01-09 09:15:00+05:30,18845.90,18845.90,18801.70,18801.70
1,2015-01-09 09:16:00+05:30,18801.70,18806.05,18790.20,18794.65
2,2015-01-09 09:17:00+05:30,18796.00,18829.00,18794.65,18827.30
3,2015-01-09 09:18:00+05:30,18827.30,18864.00,18827.30,18862.10
4,2015-01-09 09:19:00+05:30,18862.10,18862.10,18843.55,18849.25


In [12]:
#df1["time"] = list(map(lambda x: x[18:],df1["ins_date"].astype(str)))
df1["date"] = list(map(lambda x: x[:19],df1["ins_date"].astype(str)))

In [13]:
df1["time"] = list(map(lambda x: x[11:],df1["date"].astype(str)))
df1["date1"] = list(map(lambda x: x[:10],df1["date"].astype(str)))


In [14]:
df1.tail()

,ins_date,open,high,low,close,date,time,date1
572588,2021-03-19 15:25:00+05:30,34050.90,34070.15,34038.95,34069.80,2021-03-19 15:25:00,15:25:00,2021-03-19
572589,2021-03-19 15:26:00+05:30,34069.10,34097.40,34067.20,34094.35,2021-03-19 15:26:00,15:26:00,2021-03-19
572590,2021-03-19 15:27:00+05:30,34092.80,34098.50,34084.20,34089.15,2021-03-19 15:27:00,15:27:00,2021-03-19
572591,2021-03-19 15:28:00+05:30,34085.75,34103.70,34083.80,34103.70,2021-03-19 15:28:00,15:28:00,2021-03-19
572592,2021-03-19 15:29:00+05:30,34096.90,34124.30,34095.50,34115.10,2021-03-19 15:29:00,15:29:00,2021-03-19


In [42]:
df2=df1.loc[df1['time']== '10:15:00']
df2=df2.reset_index()
df2 =df2[['date', 'open', 'high', 'low', 'close', 'time', 'date1']]

In [43]:
#df3=df1.loc[df1['time']== '10:00:00']
#df3=df3.reset_index()
#df3 =df3[['date', 'open', 'high', 'low', 'close', 'time', 'date1']]

In [44]:
df2.tail()

,date,open,high,low,close,time,date1
1521,2021-03-15 10:15:00,34795.20,34795.20,34736.10,34738.95,10:15:00,2021-03-15
1522,2021-03-16 10:15:00,35110.15,35116.15,35096.65,35114.85,10:15:00,2021-03-16
1523,2021-03-17 10:15:00,34680.00,34692.05,34665.10,34665.10,10:15:00,2021-03-17
1524,2021-03-18 10:15:00,34599.80,34635.60,34596.50,34635.00,10:15:00,2021-03-18
1525,2021-03-19 10:15:00,33795.70,33831.85,33790.35,33809.60,10:15:00,2021-03-19


In [53]:
#close_HEIKIN
n=0
df2['close_HEIKIN']=np.nan
for i in range(n,len(df2)):
  df2['close_HEIKIN'][i] =df2['close'][i]+df2['high'][i]+df2['low'][i]+df2['open'][i]
n=0
m=4
for i in range(n,len(df2)):
  df2['close_HEIKIN'][i] =df2['close_HEIKIN'][i]/m

#open_aHEIKIN
n=1
n=0
df2['open_HEIKIN']=np.nan
for i in range(n,len(df2)):
  df2['open_HEIKIN'][0] =df2['open'][0]
n=1
for i in range(n,len(df2)):
  df2['open_HEIKIN'][i] =df2['open'][i-1]+df2['close'][i-1]
n=1
m=2
for i in range(n,len(df2)):
  df2['open_HEIKIN'][i] =df2['open_HEIKIN'][i]/m

#high_HEIKIN and low_HEIKIN
df2['high_HEIKIN']=np.nan
df2['high_HEIKIN'] = df2[['open_HEIKIN','close_HEIKIN','high']].max(axis=1)
df2['low_HEIKIN']=np.nan
df2['low_HEIKIN'] = df2[['open_HEIKIN','close_HEIKIN','low']].min(axis=1)

In [54]:
df2.head()

,date,open,high,low,close,time,date1,close_HEIKIN,open_HEIKIN,high_HEIKIN,low_HEIKIN
0,2015-01-09 10:15:00,18781.15,18792.20,18781.15,18786.70,10:15:00,2015-01-09,18785.3000,18781.150,18792.200,18781.150
1,2015-01-12 10:15:00,18748.15,18752.65,18742.80,18752.65,10:15:00,2015-01-12,18749.0625,18783.925,18783.925,18742.800
2,2015-01-13 10:15:00,18889.90,18889.90,18882.10,18884.85,10:15:00,2015-01-13,18886.6875,18750.400,18889.900,18750.400
3,2015-01-14 10:15:00,18684.65,18684.95,18677.35,18681.00,10:15:00,2015-01-14,18681.9875,18887.375,18887.375,18677.350
4,2015-01-15 10:15:00,19084.15,19088.05,19073.35,19081.65,10:15:00,2015-01-15,19081.8000,18682.825,19088.050,18682.825


In [55]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1526 non-null   object 
 1   open          1526 non-null   object 
 2   high          1526 non-null   object 
 3   low           1526 non-null   object 
 4   close         1526 non-null   object 
 5   time          1526 non-null   object 
 6   date1         1526 non-null   object 
 7   close_HEIKIN  1526 non-null   float64
 8   open_HEIKIN   1526 non-null   float64
 9   high_HEIKIN   1526 non-null   float64
 10  low_HEIKIN    1526 non-null   float64
dtypes: float64(4), object(7)
memory usage: 131.3+ KB


In [56]:
df2['open'] = df2['open'].astype('float')
df2['high'] = df2['high'].astype('float')
df2['low'] = df2['low'].astype('float')
df2['close'] = df2['close'].astype('float')

In [57]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1526 non-null   object 
 1   open          1526 non-null   float64
 2   high          1526 non-null   float64
 3   low           1526 non-null   float64
 4   close         1526 non-null   float64
 5   time          1526 non-null   object 
 6   date1         1526 non-null   object 
 7   close_HEIKIN  1526 non-null   float64
 8   open_HEIKIN   1526 non-null   float64
 9   high_HEIKIN   1526 non-null   float64
 10  low_HEIKIN    1526 non-null   float64
dtypes: float64(8), object(3)
memory usage: 131.3+ KB


In [58]:
df2['candle_type']=np
n=0
for i in range(n, len(df2)):
  if df2['open_HEIKIN'][i]<df2['close_HEIKIN'][i]:
    df2['candle_type'][i]='Green'
  else:
    df2['candle_type'][i]='Red'

In [59]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526 entries, 0 to 1525
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1526 non-null   object 
 1   open          1526 non-null   float64
 2   high          1526 non-null   float64
 3   low           1526 non-null   float64
 4   close         1526 non-null   float64
 5   time          1526 non-null   object 
 6   date1         1526 non-null   object 
 7   close_HEIKIN  1526 non-null   float64
 8   open_HEIKIN   1526 non-null   float64
 9   high_HEIKIN   1526 non-null   float64
 10  low_HEIKIN    1526 non-null   float64
 11  candle_type   1526 non-null   object 
dtypes: float64(8), object(4)
memory usage: 143.2+ KB


In [62]:
df2.tail()

,date,open,high,low,close,time,date1,close_HEIKIN,open_HEIKIN,high_HEIKIN,low_HEIKIN,candle_type
1521,2021-03-15 10:15:00,34795.20,34795.20,34736.10,34738.95,10:15:00,2021-03-15,34766.3625,36268.750,36268.75,34736.100,Red
1522,2021-03-16 10:15:00,35110.15,35116.15,35096.65,35114.85,10:15:00,2021-03-16,35109.4500,34767.075,35116.15,34767.075,Green
1523,2021-03-17 10:15:00,34680.00,34692.05,34665.10,34665.10,10:15:00,2021-03-17,34675.5625,35112.500,35112.50,34665.100,Red
1524,2021-03-18 10:15:00,34599.80,34635.60,34596.50,34635.00,10:15:00,2021-03-18,34616.7250,34672.550,34672.55,34596.500,Red
1525,2021-03-19 10:15:00,33795.70,33831.85,33790.35,33809.60,10:15:00,2021-03-19,33806.8750,34617.400,34617.40,33790.350,Red


In [65]:
df2.to_excel("HEIKIN ASHI Candle1.xlsx")